# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [10]:
workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, sep = '\n')

udacityws
udacitycapstone
japaneast


In [68]:
# choose a name for experiment
experiment_name = 'capstone'
experiment=Experiment(workspace, experiment_name)

dataset = Dataset.get_by_name(workspace, name='Census Income Data Set')
df = dataset.to_pandas_dataframe()
df.head(20)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.00,Never-married,Adm-clerical,Not-in-family,White,Male,2174.00,0.00,40.00,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.00,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00,0.00,13.00,United-States,<=50K
2,38,Private,HS-grad,9.00,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.00,0.00,40.00,United-States,<=50K
3,53,Private,11th,7.00,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.00,0.00,40.00,United-States,<=50K
4,28,Private,Bachelors,13.00,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.00,0.00,40.00,Cuba,<=50K
5,37,Private,Masters,14.00,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.00,0.00,40.00,United-States,<=50K
6,49,Private,9th,5.00,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.00,0.00,16.00,Jamaica,<=50K
7,52,Self-emp-not-inc,HS-grad,9.00,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00,0.00,45.00,United-States,>50K
8,31,Private,Masters,14.00,Never-married,Prof-specialty,Not-in-family,White,Female,14084.00,0.00,50.00,United-States,>50K
9,42,Private,Bachelors,13.00,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.00,0.00,40.00,United-States,>50K


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [16]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6, 
                                                           idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(workspace, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [29]:
# TODO: Put your automl settings here
automl_settings = {
    "compute_target": cpu_cluster,
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(
    task = 'classification',
    training_data = dataset,
    #valudation_data = validation_data,
    label_column_name = 'income',
    n_cross_validations = 5,
    **automl_settings)

In [30]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone,AutoML_8b7c9872-e139-4e02-a108-1643a9d139e7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Generating individually featurized CV splits.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected

In [31]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [32]:
automl_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
capstone,AutoML_8b7c9872-e139-4e02-a108-1643a9d139e7,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_8b7c9872-e139-4e02-a108-1643a9d139e7',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-18T11:45:17.231656Z',
 'endTimeUtc': '2021-07-18T12:27:03.829253Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone","subscription_id":"653662f1-95de-4498-b876-1fd625bf5d18","resource_group":"udacitycapstone","workspace_name":"udacityws","region":"japaneast","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classificat

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [33]:
best_automl_run, fitted_automl_model = automl_run.get_output()

print(fitted_automl_model)
fitted_automl_model._final_estimator

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=2.3958333333333335, subsample=1, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.4, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.13333333333333333, 0.06666666666666667]))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


PreFittedSoftVotingClassifier(
    estimators=[('46', Pipeline(memory=None,
             steps=[('standardscalerwrapper',
                     StandardScalerWrapper(
        copy=True,
        with_mean=False,
        with_std=False
    )),
                    ('xgboostclassifier',
                     XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.2, gamma=0.01, grow_policy='lossguide', max_bin=255, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo...
        iteration_timeout_mode=0,
        iteration_timeout_param=None,
        feature_column_names=None,
        label_column_name=None,
        weight_column_name=None,
        cv_split_column_names=None,
        enable_streaming=None,
        timeseries_param_dict=None,
        gpu_training_param_dict={'processing_unit_type': 'cpu'}
    ), random_state=0, reg_alpha=0, reg_lambda=1.7708333333333335, subsample=1, tree_method='hist'))],
             verbose=False))

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [34]:
print(best_automl_run)

Run(Experiment: capstone,
Id: AutoML_8b7c9872-e139-4e02-a108-1643a9d139e7_68,
Type: azureml.scriptrun,
Status: Completed)


In [35]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_automl_model, './automl_best_model.joblib', compress = 3)

['./automl_best_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [58]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

# Register the model to deploy
model = automl_run.register_model(description = 'Census Income Data Set')
print(f'model ID {automl_run.model_id}')
# Combine scoring script & environment in Inference configuration
env = best_automl_run.get_environment().save_to_directory(path='environments')
script_file= 'score.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)
inference_config = InferenceConfig(entry_script=script_file, environment=env)

model ID AutoML8b7c9872e68


In [59]:
from azureml.core.webservice import AciWebservice, Webservice

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [60]:
from azureml.core.model import Model
# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = workspace,
    name = "capstone-model",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite = True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-18 20:35:16+00:00 Creating Container Registry if not exists.
2021-07-18 20:35:19+00:00 Use the existing image.
2021-07-18 20:35:20+00:00 Generating deployment configuration.
2021-07-18 20:35:21+00:00 Submitting deployment to compute..
2021-07-18 20:35:24+00:00 Checking the status of deployment capstone-model..
2021-07-18 20:40:49+00:00 Checking the status of inference endpoint capstone-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [61]:
print(service.state)

Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [67]:
import json

input_payload = json.dumps({
    'data': json.loads(df.drop('income', axis = 1)[0:20].to_json(orient='records'))
})

output = service.run(input_payload)

print(output)

{"result": ["<=50K", "<=50K", "<=50K", "<=50K", "<=50K", ">50K", "<=50K", "<=50K", ">50K", ">50K", ">50K", ">50K", "<=50K", "<=50K", "<=50K", "<=50K", "<=50K", "<=50K", "<=50K", ">50K"]}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()